<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/drivendata/flu%20shot%20learning/flu_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

In [34]:
train_x = pd.read_csv('/content/drive/MyDrive/data/drivendata_flu shot /train_x.csv', index_col=0)
train_y = pd.read_csv('/content/drive/MyDrive/data/drivendata_flu shot /train_y.csv', index_col=0)
test = pd.read_csv('/content/drive/MyDrive/data/drivendata_flu shot /test.csv', index_col=0)
submission = pd.read_csv('/content/drive/MyDrive/data/drivendata_flu shot /submission.csv', index_col=0)

In [20]:
train = pd.concat([train_x, train_y], axis=1)
train['num_of_vacc'] = train['h1n1_vaccine'] + train['seasonal_vaccine']

# A view of data

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

In [27]:
train

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine,num_of_vacc
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1,1
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0,0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,1.0,5.0,2.0,2.0,65+ Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN,0,0,0
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea,0,0,0
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,4.0,2.0,5.0,4.0,2.0,55 - 64 Years,Some College,White,Female,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN,0,1,1


In [48]:
train['employment_status'].value_counts()

Employed              13560
Not in Labor Force    10231
Unemployed             1453
Name: employment_status, dtype: int64

# Notes


1.   `household_adults` and `household_children` have `nan` in the same rows.
2.   `age group`, `sex`, `race`, `hhs_geo_region` and `census_msa` have no `nan`
3.   


In [30]:
train.groupby('doctor_recc_h1n1')['h1n1_vaccine'].value_counts()

doctor_recc_h1n1  h1n1_vaccine
0.0               0               16529
                  1                2610
1.0               1                2879
                  0                2529
Name: h1n1_vaccine, dtype: int64

In [32]:
train.groupby('doctor_recc_seasonal')['seasonal_vaccine'].value_counts()

doctor_recc_seasonal  seasonal_vaccine
0.0                   0                   10756
                      1                    5697
1.0                   1                    5977
                      0                    2117
Name: seasonal_vaccine, dtype: int64

In [29]:
train.groupby('h1n1_concern')['h1n1_knowledge'].value_counts()

h1n1_concern  h1n1_knowledge
0.0           1.0               1534
              2.0               1071
              0.0                663
1.0           1.0               4871
              2.0               2673
              0.0                581
2.0           1.0               5913
              2.0               3994
              0.0                639
3.0           1.0               2257
              2.0               1739
              0.0                574
Name: h1n1_knowledge, dtype: int64

In [28]:
train['census_msa'].unique()

array(['Non-MSA', 'MSA, Not Principle  City', 'MSA, Principle City'],
      dtype=object)

In [37]:
train['age_group'].value_counts()

65+ Years        6843
55 - 64 Years    5563
45 - 54 Years    5238
18 - 34 Years    5215
35 - 44 Years    3848
Name: age_group, dtype: int64

In [38]:
train['race'].value_counts()

White                21222
Black                 2118
Hispanic              1755
Other or Multiple     1612
Name: race, dtype: int64

In [39]:
train['income_poverty'].value_counts()

<= $75,000, Above Poverty    12777
> $75,000                     6810
Below Poverty                 2697
Name: income_poverty, dtype: int64

In [40]:
train['education'].value_counts()

College Graduate    10097
Some College         7043
12 Years             5797
< 12 Years           2363
Name: education, dtype: int64

# Preprocessing

In [36]:
train = train.drop(columns=['hhs_geo_region', 'employment_industry', 'employment_occupation'])
test = test.drop(columns=['hhs_geo_region', 'employment_industry', 'employment_occupation'])

In [50]:
arg_age_group = {'65+ Years': 70, '55 - 64 Years': 60, '45 - 54 Years': 50, '35 - 44 Years': 40, '18 - 34 Years': 26}
arg_income = {'Below Poverty': 1, '<= $75,000, Above Poverty': 2, '<= $75,000, Above Poverty': 3}
arg_education = {'< 12 Years': 1, '12 Years': 2, 'Some College': 3, 'College Graduate': 4}
arg_employment = {'Unemployed': 1, 'Not in Labor Force': 2, 'Employed': 3}
label_enc = ['sex', 'marital_status', 'rent_or_own']
onehot_enc = ['race', 'census_msa']

train['age_group'] = train['age_group'].map(arg_age_group)
test['age_group'] = test['age_group'].map(arg_age_group)
train['income_poverty'] = train['income_poverty'].map(arg_income)
test['income_poverty'] = test['age_group'].map(arg_income)
train['education'] = train['education'].map(arg_education)
test['education'] = test['education'].map(arg_education)
train['employment_status'] = train['employment_status'].map(arg_employment)
test['employment_status'] = test['employment_status'].map(arg_employment)

In [51]:
from sklearn.preprocessing import LabelEncoder

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

In [41]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
train1 = train
test1 = test

iterimp = IterativeImputer()
train1 = pd.DataFrame(iterimp.fit_transform(train1), columns=train1.columns, index=train.index)
test1 = pd.DataFrame(iterimp.transform(test1), columns=test1.columns, index=test1.index)